# 청커 실습 20250608
청킹 방법의 종류
1. 길이 기반 텍스트 토큰화 
    - langchain_text_splitters.CharacterTextSplitter
    - 간단한 구현/일관된 청크 크기/다양한 모델 요구 사항에 쉽게 적응 가능
    - 토큰 기반 : 언어 모델로 작업할 때 유용한 토큰 개수에 따라 텍스트를 분할합니다.(tictoken으로 토큰 길이 셀 수 있음-토큰제한이 있으므로 토큰을 세봐야함)    
    - 문자 기반 : 텍스트를 문자 수에 따라 분할하므로 다양한 유형의 텍스트에서 일관성을 유지할 수 있음(가장 쉬운방법-띄어쓰기 또는 \n 기준으로 분할함)
2. 텍스트 구조 기반 토큰화 (문단을 유지하기 위함)
    - langchain_text_splitters.RecursiveCharacterTextSpliiter
    - 더 큰 단위(예: 문단)를 그대로 유지하기 위함
    - 단위가 청크 크기를 초과하면 다음 레벨(예: 문장)로 이동
    - 필요한 경우 이 과정은 단어 수준까지 계속됨
3. 문서 구조 기반 토큰화 
    - HTML, 마크다운, JSON 파일과 같이 고유한 구조를 가진 문서의 경우, 의미적으로 관련된 텍스트가 자연스럽게 그룹화되는 경우가 많으므로 구조에 따라 문서를 분할하는 것이 좋음
    - 문서의 논리적 구성을 유지/각 청크 내에서 컨텍스트를 유지/검색이나 요약과 같은 다운스트림 작업에 더 효과적
        - 컨텍스트 유지: \<h2>(제목)과 \<p>(내용)를 하나의 청크로 묶음으로써, “이 모델”이 무엇을 지칭하는지 명확해짐.
    - 문서 기반 토큰화 예시
        - 마크다운 : 헤더를 기준으로 분할(예: #, ##, ###)
        - HTML : 태그를 사용하여 분할
        - JSON : 객체 또는 배열 요소로 분할
        - 코드 : 함수, 클래스 또는 논리 블록으로 구분

출처 : https://python.langchain.com/docs/concepts/text_splitters/#document-structured-based

In [ ]:
!pip install langchain
!pip install numpy
!pip install pandas
!pip install pymupdf
!pip install tiktoken
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.6 MB/s eta 0:00:00a 0:00:01


In [47]:
import pandas as pd
import numpy as np
import fitz

from langchain.text_splitter import CharacterTextSplitter # 길이 기반 청커
from langchain_text_splitters import RecursiveCharacterTextSplitter # 텍스트 구조 기반 청커
from langchain.schema import Document # LangChain Document 객체 리스트 형태로 변환
from langchain_community.document_loaders import PyMuPDFLoader
import tiktoken

### bring pdf to text

In [96]:
import fitz  # pip install pymupdf

def load_pdf_text(filepath):
    doc = fitz.open(filepath)
    all_text = ""
    for page in doc:
        all_text += page.get_text()
    doc.close()
    return all_text

pdf_path = "../dataset/240806_다자녀를_둔_부모라면_꼭_알아야_하는_법령을_소개합니다_최종.pdf"
text = load_pdf_text(pdf_path)
print(text)

보도자료
보도시점
즉시
배포
2024. 8. 7.(수)
다자녀를 둔 부모라면 꼭 알아야 하는
법령을 소개합니다
 최근 저출생 기조에 더욱 적극적으로 대응하기 위해 다자녀 가구에 대한 지
원이 확대되는 추세이다. 특히 다자녀 가구의 기준도 기존 3자녀 이상에서 2
자녀 이상으로 완화되고 있다. 다자녀를 둔 부모들이 꼭 알아야 할 법령과 제
도 몇 가지를 소개한다. 
 다둥이 임신ㆍ출산진료비 이용권과 첫만남 이용권, 각종 세금 감면 혜택
 「국민건강보험법」에 따라 지원하는 ‘임신ㆍ출산진료비 이용권’은 기존 단
태아(單胎兒) 100만원, 다태아(多胎兒) 140만원에서 태아당 100만원으로 증
액되어, 쌍둥이인 경우 60만원을 더 받게 되었다. ‘임신ㆍ출산진료비 이용권’은 
출산(예정)일부터 2년 이내에 사용해야 한다. 「저출산ㆍ고령사회기본법」에 
따라 출산 후 지급되는 ‘첫만남 이용권’은 2024년부터 둘째 자녀 이상인 경우 
기존 200만원에서 300만원으로 상향되었다. 따라서 쌍둥이는 첫째 200만원, 
둘째 300만원으로 총 500만원을 지급받을 수 있다. ‘첫만남 이용권’은 아이의 
출생일부터 1년 이내에 사용해야 한다.
 「소득세법」에 따른 자녀세액공제 혜택도 있다. 종합소득세를 신고하는 사
람의 기본공제 대상자에 해당하는 8세 이상의 자녀나 손자녀의 수에 따라 종
합소득산출세액을 공제받을 수 있다. 자녀 1명의 경우 연 15만원, 자녀 2명은 
연 35만원, 자녀가 3명 이상이면 연 35만원과 2명을 초과하는 1명당 연 30
만원을 합한 금액까지 공제된다. 또한 해당 과세기간에 출산하거나 입양한 공
제대상 자녀가 첫째인 경우 연 30만원, 둘째인 경우 연 50만원, 셋째 이상인 
경우 연 70만원을 세액공제 받을 수 있다. 연말정산 시 근로소득 원천징수영
수증에 해당되는 자녀세액공제항목을 기입하여 제출하면 된다.
 「지방세특례제한법」에 따라 자동차 취득세도 할인 또는 면제받을 수 있다. 
18세 미만의 자녀를 3명 이상 양육하는 사람이 양육을 목적으로 2

### charactertextspliiter로 text 청킹

In [116]:
# 길이기반 
# cl100k_base : OpenAI의 토크나이저(tokenizer) 중 하나로, 특히 GPT-4, GPT-3.5-turbo, text-embedding-ada-002 등 최신 모델에서 사용되는 기본 토큰 인코딩 방식
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=50, chunk_overlap=10
)
splitted_text = text_splitter.split_text(text)

In [ ]:
len(splitted_text[1])

In [114]:
len(text)

3317

In [119]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
tokens = encoding.encode(text)
print("🔍 총 토큰 수:", len(tokens))

🔍 총 토큰 수: 3140


In [125]:
# text로 할때의 separator가 문제여서 spliited되지 않았었음! 문자열 기반이기 때문
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=250,
    chunk_overlap=50,
    separator="\n"  # 또는 "." 또는 " " 등 - separator 가 \n\n이어서 그런거같다!
)
splitted = text_splitter.split_text(text)
for i, chunk in enumerate(splitted):
    print(f"--- Chunk {i+1} (Length: {len(chunk)}) ---\n{chunk[:100]}...\n")

--- Chunk 1 (Length: 244) ---
보도자료
보도시점
즉시
배포
2024. 8. 7.(수)
다자녀를 둔 부모라면 꼭 알아야 하는
법령을 소개합니다
 최근 저출생 기조에 더욱 적극적으로 대응하기 위해 다자녀 가구에 대...

--- Chunk 2 (Length: 212) ---
다둥이 임신ㆍ출산진료비 이용권과 첫만남 이용권, 각종 세금 감면 혜택
 「국민건강보험법」에 따라 지원하는 ‘임신ㆍ출산진료비 이용권’은 기존 단
태아(單胎兒) 100만원, 다태아(多...

--- Chunk 3 (Length: 243) ---
출산(예정)일부터 2년 이내에 사용해야 한다. 「저출산ㆍ고령사회기본법」에 
따라 출산 후 지급되는 ‘첫만남 이용권’은 2024년부터 둘째 자녀 이상인 경우 
기존 200만원에서 3...

--- Chunk 4 (Length: 260) ---
「소득세법」에 따른 자녀세액공제 혜택도 있다. 종합소득세를 신고하는 사
람의 기본공제 대상자에 해당하는 8세 이상의 자녀나 손자녀의 수에 따라 종
합소득산출세액을 공제받을 수 있다...

--- Chunk 5 (Length: 251) ---
제대상 자녀가 첫째인 경우 연 30만원, 둘째인 경우 연 50만원, 셋째 이상인 
경우 연 70만원을 세액공제 받을 수 있다. 연말정산 시 근로소득 원천징수영
수증에 해당되는 자녀...

--- Chunk 6 (Length: 232) ---
월 31일까지 자동차를 취득하여 등록하는 경우 1대에 한하여 취득세를 면제
받을 수 있다. 자동차 종류별로 감면액의 한도가 다르며, 7인 미만 승용자동
차는 140만원까지 취득세를...

--- Chunk 7 (Length: 217) ---
「국민연금법」에 따라 국민연금 가입 기간을 자녀 수에 따라 추가로 인정
해주는 ‘출산 크레딧’ 제도를 시행하고 있다. 2008년 1월 1일 이후 둘째 자
녀를 출산한 2자녀 이상인...

--- Chunk 8 (Length: 229) ---
녀가 2명인 경우 12개월, 3명 이상인 경우

In [127]:
len(splitted) # 청크 개수 17개로 쪼개짐

17

In [133]:
for i in range(0,5):
    print(f"=={i}==")
    print(splitted[i])
    print()

==0==
보도자료
보도시점
즉시
배포
2024. 8. 7.(수)
다자녀를 둔 부모라면 꼭 알아야 하는
법령을 소개합니다
 최근 저출생 기조에 더욱 적극적으로 대응하기 위해 다자녀 가구에 대한 지
원이 확대되는 추세이다. 특히 다자녀 가구의 기준도 기존 3자녀 이상에서 2
자녀 이상으로 완화되고 있다. 다자녀를 둔 부모들이 꼭 알아야 할 법령과 제
도 몇 가지를 소개한다. 
 다둥이 임신ㆍ출산진료비 이용권과 첫만남 이용권, 각종 세금 감면 혜택

==1==
다둥이 임신ㆍ출산진료비 이용권과 첫만남 이용권, 각종 세금 감면 혜택
 「국민건강보험법」에 따라 지원하는 ‘임신ㆍ출산진료비 이용권’은 기존 단
태아(單胎兒) 100만원, 다태아(多胎兒) 140만원에서 태아당 100만원으로 증
액되어, 쌍둥이인 경우 60만원을 더 받게 되었다. ‘임신ㆍ출산진료비 이용권’은 
출산(예정)일부터 2년 이내에 사용해야 한다. 「저출산ㆍ고령사회기본법」에

==2==
출산(예정)일부터 2년 이내에 사용해야 한다. 「저출산ㆍ고령사회기본법」에 
따라 출산 후 지급되는 ‘첫만남 이용권’은 2024년부터 둘째 자녀 이상인 경우 
기존 200만원에서 300만원으로 상향되었다. 따라서 쌍둥이는 첫째 200만원, 
둘째 300만원으로 총 500만원을 지급받을 수 있다. ‘첫만남 이용권’은 아이의 
출생일부터 1년 이내에 사용해야 한다.
 「소득세법」에 따른 자녀세액공제 혜택도 있다. 종합소득세를 신고하는 사

==3==
「소득세법」에 따른 자녀세액공제 혜택도 있다. 종합소득세를 신고하는 사
람의 기본공제 대상자에 해당하는 8세 이상의 자녀나 손자녀의 수에 따라 종
합소득산출세액을 공제받을 수 있다. 자녀 1명의 경우 연 15만원, 자녀 2명은 
연 35만원, 자녀가 3명 이상이면 연 35만원과 2명을 초과하는 1명당 연 30
만원을 합한 금액까지 공제된다. 또한 해당 과세기간에 출산하거나 입양한 공
제대상 자녀가 첫째인 경우 연 30만원, 둘째인 경우 연 50만원, 셋째 이상인

==4==
제대상

### recursivecharactersplitter 로 text 청킹

In [ ]:

# load pdf to langchain documents
pdf_path = "../dataset/240806_다자녀를_둔_부모라면_꼭_알아야_하는_법령을_소개합니다_최종.pdf"
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()  # -> List[Document]

for i, doc in enumerate(documents):
    print(f"[Doc {i+1}] Length: {len(doc.page_content)} characters")